### Computer vision task

 ## COCO
- один из наиболее популярных датасатов содержащий данные для сегментации и детектирования.

- categoryes
- masks
- bounding boxes
- captions
- person_keypoints
...

https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocoDemo.ipynb

In [ ]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip

Для работы с датасетом используется пакет `pycocotools`

В COCO 90 категорий объектов

https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch

In [ ]:
from pycocotools.coco import COCO

coco=COCO('annotations/instances_val2017.json')

cat_ids = coco.getCatIds()
print("Categories count",len(cat_ids))
print(cat_ids)

0 - не используется в качестве номера категории. Обычно его используют для обозначения класса фона.

In [ ]:
# display COCO categories and supercategories

cats = coco.loadCats(coco.getCatIds())

nms=[str(cat['id']) + ":"  + cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))



Есть так же суперкатегории

In [ ]:
print(cats[2])
print(cats[3])


nms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(nms)))

Датесет большой, поэтому удобно выгружать данные частями

In [ ]:
# get all images containing given categories, select one at random
catIds = coco.getCatIds(catNms=['person','cat']); # person and cat
imgIds = coco.getImgIds(catIds=catIds );
#imgIds = coco.getImgIds() #imgIds = [324158]
print("Total images with person and cat ",len(imgIds))
print(imgIds)
img_list = coco.loadImgs(imgIds[0])
img = img_list[0]
print("Image data", img)

Load image

In [ ]:
import skimage.io as io
import matplotlib.pyplot as plt

I = io.imread(img['coco_url'])
plt.axis('off')
plt.imshow(I)
plt.show()

Конвертация в PIL формат

In [ ]:
from PIL import Image
import requests
from io import BytesIO

response = requests.get(img['coco_url'])
pil_img = Image.open(BytesIO(response.content))
plt.imshow(pil_img)

Информация о разметке

https://cocodataset.org/#format-data


```
  "segmentation" : RLE or [polygon],
  "area" : float,
  "bbox" : [x,y,width,height],
  "iscrowd" : 0 or 1,
```

Полигон это набор координат [x1,y1, x2,y2 ... ]

объект может описываться несколькими полигонами

In [ ]:
# load and display instance annotations
plt.imshow(I); plt.axis('off')
annIds = coco.getAnnIds(imgIds=img['id'])
anns = coco.loadAnns(annIds)

def dump_anns(anns):
  for i, a in enumerate(anns):
    print(f"#{i}")
    for k in a.keys():
      print(k,": ",a[k])
dump_anns(anns)
coco.showAnns(anns)

Что такое [RLE](https://en.wikipedia.org/wiki/Run-length_encoding) ?

run-length encoding

https://www.youtube.com/watch?v=h6s61a_pqfM

In [ ]:
plt.rcParams["figure.figsize"] = (160,80)
print(catIds)
annIds = coco.getAnnIds(catIds=catIds, iscrowd = True)
anns = coco.loadAnns(annIds[0:1])

dump_anns(anns)

img = coco.loadImgs(anns[0]['image_id'])[0]
I = io.imread(img['coco_url'])
plt.imshow(I); #plt.axis('off')
coco.showAnns(anns) # People in the stands 
seg = anns[0]['segmentation']

print('Counts',len(seg['counts']))
print('Size',seg['size'])

FCN

https://pytorch.org/hub/pytorch_vision_fcn_resnet101/

In [ ]:
import torchvision

torchvision.models.segmentation.fcn_resnet50(pretrained=False, progress=True, num_classes=21, aux_loss=None, **kwargs)

DeepLab

COCO mAP

Поговорим о зрении человека. Поскольку зрение относится к области искусственного интеллекта, значит, наиболее известный эталон – зрение человека. По оценкам, 25% мозга занято решением задачи зрения. С помощью глаз мы извлекаем визуальные подсказки, какие-то характеристики реальных объектов, сопоставляем их с наши априорными знаниями об устройстве окружающего мира и делаем выводы.

Мы можем распознать объекты на изображении, используя следующие подсказки:
1) Цвет

2) Контур

3) Текстура

4) Тени и освещение

5) Контекст (окружение объекта) 

К априорным знаниям об объекте можно отнести следующее:
1) Размеры

2) Контекст

3) Типичный диапазон распределения конкретной характеристики

4) Эталонные примеры

## Чулан

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-005.png" width="700">

### Semantic segmentation

ЗАдача сегментации - понять где именно на изображении располагаются объекты. С одной стороны может быть интересно где находится объект (область в границе) в грубом приближении. С другой стороны есть задача - выделить гарницы объхекта более четко, где нас интересует не область, а пиксельная маска. 

В общем случае задача сегментации – разделить изображение на фрагменты (группы пикселей) по некоторому общему критерию. В зависимости от критерия получаются разные задачи сегментации изображения.

1. Извлечение объекта – выделение конкретного произвольного объекта, указанного пользователем или по-другому заданного. Например, пользователь может выделить объект ограничивающим прямоугольником или нарисовать контур объекта.
2. Сегментация без учителя – разделение изображения на регионы, однородные по своим визуальным характеристикам и отличающиеся от соседних регионов. Например, нужно выделить области, соответствующие разным фрагментам фотографии котика (земля, небо, котик)

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-006.png" width="700">

Поскольку такие методы разбивают объекты обычно на несколько сегментов, то их еще называют методами пересегментации, потому что мы сегментируем чрезмерно по сравнению с тем, как хотелось бы. Другой вариант – семантическая сегментация.

Семантическая сегментация – попиксельная разметка изображения, где каждая метка соответствует определенному объекту. 

При этом:
* Разные пиксели одного объекта существенно отличаются друг от друга по признакам (яркости, цвету, текстуре окрестности)
* Единственное, что у них общее – «семантика»
* Поэтому задача сегментации тесно связана с задачей распознавания

### The problem

**Оценка точности сегментации**

Как оценивается точность сегментации? Во-первых, есть попиксельные метрики, то есть мы оцениваем долю правильно отсегментированных пикселей на изображении независимо от того, какую задачу мы рассматриваем. Если метка пикселя верна – это
хорошо, и рассматриваем долю верных меток пикселей по отношению ко всем пикселям.


Второй вариант оценки примерно такой же, как мы использовали в детектировании объектов, то есть у нас имеется множество сегментов, мы сопоставляем сегменты друг другу и считаем критерий IOU (intersection over union) (рис. 98). Мы для всех сегментов считаем долю правильно классифицированных пикселей по отношению к комбинации правильно классифицированных, ложно позитивных срабатываний и ложноотрицательных срабатываний для каждого рассматриваемого класса объектов

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-115.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-007.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-009.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-010.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-012.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-013.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-015.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-017.png" width="700">

### In-Network upsampling: Max Unpooling

Одним из направлений было исследование новых способов повышения разрешения карт признаков с использованием какой-то дополнительной информацией. Так получился слой max unpooling 

Вспомним, что при понижении разрешения изображения мы обычно используем max pooling, например, с шагом 2х2, то есть из каждого блока 2х2 пикселя мы сохраняем только одно максимальное значение. Запомним индекс, из которого было получено максимальное значение, поскольку нам важно знать, где именно в картинке было это максимальное значение. Если мы запомним этот индекс, тогда сможем выполнить операцию повышения разрешения, которая будет не просто делать пиксели с равномерным шагом, а заносить целевой пиксель именно туда, откуда мы взяли изначально максимальное значение. Уже после этого будем проходиться по этой карте
набором сверток, которые будут интерполировать промежуточные значения.

* Сохраняем индексы каждого max pooling слоя
* При повышении разрешения копируем значения из выхода max pooling слоя с учетом запомненных индексов, применяем обученные свертки для сглаживания

С помощью операции unpooling появилась архитектура для сегментации изображения, фактически для декодирования признаков, Encoder-Decoder with Max Unpooling

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-119.png" width="700">

Здесь между соответствующими словами кодировщика и декодировщика передаётся информация в виде индексов операции max-pooling, то есть не вся информация со слоев высокого разрешения теряется, она сохраняется и передается на декодировщик. Эта архитектура оказалась точнее предыдущей.

В одной из работ эту архитектуру довели до абсолюта, то есть построили такой кодировщик, который превращает картинку в вектор 1х1 пикселя (из картинки сохраняется только контекст). Используя сохраненные max-pooling индексы, повышаем разрешение до исходного, предсказывая карту разметки. Как показали эксперименты, контекст вообще можно проигнорировать и заменить вектор на константный, так как в индексах операции max-pooling хранится достаточно информации для того, чтобы предсказать карту сегментации для исходного изображения. Поскольку свертки связаны с визуальными образами, информации о том, где свертка дала максимальное значение, достаточно, чтобы детектировать объект. 

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-019.png" width="700">

### Learnable upsampling: Transpose convolution

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-020.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-021.gif" width="700">

### Learnable upsampling: 1D example

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-031.png" width="700">

### Convolution as matrix multiplication (1D example)

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-033.png" width="700">

### Semantic segmentation idea: fully convolutional

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-034.png" width="700">

### Semantic segmentation: summary

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-035.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-036.png" width="700">

### Object detection: single object

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-039.gif" width="700">

### Object detection: multiple objects

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-042.gif" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-044.gif" width="700">

### Cropping features: Rol Pool

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-064.gif" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-048.png" width="700">

### Instant segmentation

Следующая постановка задачи – сегментация экземпляров (Instance segmentation). Мы фиксируем классы объектов, которые хотим выделить и каждый экземпляр заданных классов мы помечаем своей меткой. Задача сегментации экземпляров – это дальнейшее развитие задачи детектирования объектов. Если в обычной задаче детектирования мы локализовывали объекты, например, типа «автомобили» и
показывали, где они расположены с помощью прямоугольной рамки, то здесь нужно указать точную область, которую занимает каждый конкретный экземпляр.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-093.png" width="700">

В задаче сегментации экземпляров мы выделяем объекты типа things, которые хорошо локализованы в пространстве. В задаче семантической сегментации мы можем пометить области, которые соответствуют другой категории объектов, таких как небо и
дорога. Разумеется, возникает задача, в которой и семантическая сегментация, и сегментация экземпляров являются подзадачами, такая задача получила название паноптическая сегментация (panoptic segmentation)

### Instant segmentation: mask R-CNN

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-095.png" width="700">

### Recap: lots of computer vision taska

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-005.png" width="700">

### Object detection

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-037.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-038.png" width="700">

### Region proposal: selective search

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-049.png" width="700">

### R-CNN

Первая нейросеть на основе каскада – Region-based convolution networks (R-CNN), по сути двухэтапный каскад (рис.77). Мы используем какой-то внешний алгоритм как генератор гипотезы объектов, с помощью него генерируем заданное количество гипотез, чтобы обеспечить достаточно высокую полноту. Затем каждую из этих гипотез мы вырезаем из изображения и подаем на вход классификатору.

Метод генерации гипотез, который использовался в R-CNN (регионально сверточных сетях), - один из методов иерархической сегментации, то есть этот метод последовательно разбивает изображение на однородные сегменты.

### Обучение R-CNN

Поскольку нейросети на тот момент принимали на вход только квадратные изображения фиксированного разрешения, а сегменты, которые находил метод были произвольных пропорций, то нам нужно было их преобразовывать. При этом в тот
момент эталонные коллекции для обучения были не очень большими по сравнению с теми коллекциями, ан которых обучались для задачи классификации. 

Поэтому примеров в этих коллекциях было недостаточно для того, чтобы обучить нейросеть с нуля. Поэтому нейросеть использовали только как метод извлечения признаков, брали выход одного из слоев, обычно полносвязного, и подавали его на вход методу опорных векторов. Фактически взяли метод HOG+CVN, но вместо того, чтобы использовать гистограмму ориентированных градиентов как признаки, брались нейросетевые признаки с сети классификатора. Это было достаточно сложно и медленно, но точность высокая.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-116.png" width="700">

При этом очень сильно повысить точность помогла еще одна идея – уточнение ограничивающего прямоугольника или регрессия bbox.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-117.png" width="700">

Суть идеи заключается в следующем:

* Исходные гипотезы bbox могут быть очень неточными, поскольку мы получаем их от внешнего метода и не знаем насколько точно они локализованы. За счет того, что нейросетевой метод устойчив к положению объектов внутри этого ограничивающего прямоугольника, даже если локализация не очень точная, он все равно выдаст правильный ответ: есть объект или нет

* Чтобы повысить точность локализации, вместо классификатора на тех же самых признаках обучим линейную регрессию, то есть мы будем находить сдвиг ограничивающего прямоугольника относительно исходного, где именно должен располагаться объект

* Такую регрессию можно легко обучить, и она существенно улучшает точность классификации


<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-118.png" width="700">

Таким образом, метод регрессии bbox позволяет взять неточную гипотезу и уточнить ее. Такого не было до нейросетевых методов, до этого мы полагались на ограничивающий прямоугольник и для него только предсказывали метку: он или не он.

Теперь мы можем не только предсказывать метку, но и уточнять ее, за счет этого мы можем просматривать гораздо меньше прямоугольников, чем раньше. Теперь мы можем брать прямоугольники, фрагменты с достаточно большим шагом на меньшем
числе масштабов за счет того, что мы будем уточнять наши изначальные гипотезы значительно.

У R-CNN было много недостатков:

1. Вычисляем нейросетевые признаки независимо для каждого окна-гипотезы. Поскольку они пересекаются, это ведет к избыточным вычислениям

2. Нужно масштабировать фрагменты-гипотез до нужного разрешения 

3. Сложная процедура обучения

4. Зависимость от внешнего алгоритма генерации гипотез

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-055.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-056.png" width="700">

### Slow R-CNN

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-057.png" width="700">

### Fast R-CNN

На основании ROI-pooling сделали модификацию метода R-CNN – Fast R-CNN.

**Основные идеи этого метода:**

Вычислять сверточные признаки по всему изображению, извлекать из конкретных гипотез вектор-признаки с помощью слоя ROI-pooling. И за тем вместо того, чтобы подавать их на вход SVM, подавать их на вход обычным полносвязным слоям нейросетевого классификатора и одновременно по этим признакам делать и классификацию с помощью soft max, и регрессию bbox. При этом мы можем обучать нашу сеть в режиме многоцелевого обучения. Таким образом, мы можем и дообучить и обучить сверточные признаки, которые мы вычисляем по всему изображению. Для того, чтобы использовать особенность этого метода, которая заключается в том, что он
не требует повторного вычисления признаков, нам нужно с каждого изображения брать много гипотез. Гипотезой в нашем случае будет какое-то окно интересов, потому что мы вычислим для этого окна интересов ошибку, затем применим обратное распространение ошибки для получения градиента по всей нейросети. Соответственно одним элементом у нас является именно регион конкретного изображения (регион интересов).


Градиенты от всей нейросети занимают много места в памяти, поэтому обычно берут малое количество изображений в одном mini-batch, но с одного изображения собирают много гипотез. Из-за огромного размера современных нейросетей сейчас mini-batch может состоять из одной картинки и нескольких десятков гипотез

Чтобы обеспечить сбалансированность выборки, нужно брать примерно одинаковое количество положительных и отрицательных гипотез внутри изображения. Если, например, мы можем в каждом изображении взять 64 гипотезы, нам нужно, чтобы 32 гипотезы из этой выборки были положительными и 32- отрицательными. Отрицательные гипотезы могут быть разными по качеству, поэтому на этом этапе
можем применить процедуру, похожую на процедуру поиска сложных примеров для обучения нейросетей.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-063.png" width="700">

### R-CNN vs Fast R-CNN

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-077.png" width="700">

### Fast-ER R-CNN

Следующий шаг, который нужно было сделать для улучшения сети – избавиться от внешнего генератора гипотез. Так появилась Faster R-CNN. Faster R-CNN – пример нейросетевой архитектуры, которая используется до сих пор. По сути Faster R-CNN = Fast R-CNN + RPN (нейросетевого генератора гипотез). 

Нейросетевой генератор гипотез – маленькая нейросеть, которая по тем же самым сверточным признакам генерирует гипотезы. Наиболее вероятные гипотезы подаются на вход ROI-pooling слою и затем классифицируются более мощным классификатором.

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-078.png" width="700">

### Region proposal network

* Маленькое скользящее окно по карте признаков (feature map)
* Маленькая нейросеть для
    - Классификации объект/не объект
    - Регрессии bbox
* Позиция окна показывает локализацию объекта относительно изображения
* Регрессия bbox показывает положение bbox относительно положения скользящего окна 

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-080.gif" width="700">

### Fast-ER R-CNN

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-085.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-086.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-087.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-089.png" width="700">

### Single-stage Object detection: YOLO/SSD/RetinaNet

Метод Faster R-CNN достаточно мощный, но зачастую нам нужны методы побыстрее. Самое медленное в этом методе – классификация гипотез с помощью полносвязного классификатора (второй этап).

Поэтому родилась следующая идея: выкинуть второй этап и сделать одностадийный метод. Для этого нужно усилить RPN. Так появились одностадийные детекторы, которые работают быстрее, но менее точно, чем Faster R-CNN. Одним из таких детекторов является YOLO (сокращение от названия статьи You Only Look Once). 

Как устроен этот детектор?

Разобьем изображение сеткой и сделаем такую нейросеть, которая будет для каждой ячейки предсказывать, есть ли объект, ограничивающий прямоугольник которого - центр данной ячейки. Фактически, разбив изображение сеткой 7х7, мы сделаем 49 гипотез. Поскольку объекты бывают разных размеров и предсказывать их одним классификатором неудобно, мы воспользуемся похожей на якори идеей и будем в каждой ячейки выдвигать 2 гипотезы: одну маленькую, а другую большую. 

В сумме наши 49 ячеек сделают 98 гипотез. Параллельно каждую ячейку заставим предсказывать класс объекта. Для каждой гипотезы у нас есть вероятность классификатора. Применим метод подавления немаксимумов жадного подавления гипотез и оставшиеся гипотезы обрезаем по порогу по степени уверенности. Если говорить о параметризации ответа (что мы хотим получить на выходе сети), то на
выходе мы хотим получить трехмерную матрицу размера 7 х 7 х (2 х 5 + 20) = 7 х 7 х 30, так как

Сетка 7 х 7, 2 гипотезы объектов, 20 классов. То есть сеть на вход берет изображение. А на выходе предсказывает 7 х 7 параметров. Эта сеть называется DarkNet.

DarkNet устроен следующим образом: он состоит из сверточных слоев таким образом, что в конце у нас появляется матрица размером 7 х 7 х 1024. Далее эту матрицу преобразуют в один полносвязный слой длиной 4096, этот слой содержит всю информацию про изображение. Из этого полносвязного слоя генерируется ответ 7 х 7 х 30

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-090.png" width="700">

Есть другой вариант, который был предложен в Google – Single Shot Detector (SSD). 

В отличие от детектора YOLO в нем нет этапа преобразования в полносвязный слой, в нем генерируются гипотезы для объектов по признакам, собранным со сверточных слоев. И в отличие от детектора YOLO, который делал одну сетку для всего изображения, в детекторе SSD генерировалось несколько масштабов, несколько разбиений изображений.

Объединив идеи одностадийного алгоритма детектирования объектов, построения пирамиды признаков и применение RPN сети по этим признакам, состоящей из 4 слоев 3 х 3 пикселей, сделали архитектуру Retina Net, которая является одной из наиболее точных архитектур для детектирования объектов в настоящее время.


RetinaNet = SSD + FPN + FocalLoss

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-020.png" width="700">

В отличие от детектора YOLO в нем нет этапа преобразования в полносвязный слой, в нем генерируются гипотезы для объектов по признакам, собранным со сверточных слоев. И в отличие от детектора YOLO, который делал одну сетку для всего изображения, в детекторе SSD генерировалось несколько масштабов, несколько разбиений изображений

### Object detection: Lots of variables...

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-091.png" width="700">

### Object detection: faster R-CNN

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-094.png" width="700">

### Mask R-CNN

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-096.png" width="700">

### Mask R-CNN: example mask training targets

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-100.png" width="700">

### Mask R-CNN: very good results

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-101.png" width="700">

### Mask R-CNN: also does pose

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-102.png" width="700">

### Beyond 2D object detection..

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-104.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-105.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-106.png" width="700">

### Dense video captioning

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-107.png" width="700">

### Objects+relationships = scene graphs

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-108.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-108.png" width="700">

### Scene graph prediction

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-109.png" width="700">

### 3D object detection

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-110.png" width="700">

### 3D object detection: simple camera model

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-111.png" width="700">

### 3D object detection: monocular camera

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-112.png" width="700">

### 3D shape protection: mesh R-CNN

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/lecture_12-113.png" width="700">

## Картинки по лекции 8 

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-03.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-07.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-08.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-09.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-10.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-11.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-12.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-14.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-15.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-16.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-17.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-19.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-20.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-21.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-22.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-23.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-24.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-25.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-26.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-27.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-28.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-29.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-30.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L8-32.png" width="700">

## Картинки по лекции 8 

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-02.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-03.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-05.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-06.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-07.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-08.png" width="500">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-09.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-10.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-11.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-12.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-13.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-14.png" width="500">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-18.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-20.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-21.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-22.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-23.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-24.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-25.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-26.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-27.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-28.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-29.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-30.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-31.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-32.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-34.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-35.png" width="700">

<img src ="https://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-1-1.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-1-2.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-42.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-43.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-44.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-45.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-46.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-47.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-48.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-49.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-50.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-51.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-52.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-54.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-55.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-56.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-57.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L12_Segmentation_Detection/img/L9-58.png" width="700">